In [1]:
import scanpy as sc
import random
from scipy.sparse import csr_matrix

## Generate CPA Norman19 split

In [ ]:
data_path = "./notebooks/neurips2025/perturbench_data/"

In [ ]:
anndata_path = f'{data_path}/norman19_cpa_hvg_normalized.h5ad'

adata = sc.read_h5ad(
    anndata_path
)
adata

AnnData object with n_obs × n_vars = 111122 × 5044
    obs: 'guide_id', 'read_count', 'UMI_count', 'coverage', 'gemgroup', 'good_coverage', 'number_of_cells', 'tissue_type', 'cell_line', 'cancer', 'disease', 'perturbation_type', 'celltype', 'organism', 'perturbation', 'nperts', 'ngenes', 'ncounts', 'percent_mito', 'percent_ribo', 'n_counts', 'condition', 'pert_type', 'cell_type', 'source', 'condition_ID', 'control', 'dose_value', 'pathway', 'cov_cond', 'pert', 'split_hardest', 'split_1', 'split_2', 'split_3', 'split_4', 'split_5', 'split_6', 'cond_harm'
    var: 'ensemble_id', 'ncounts', 'ncells', 'symbol', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'cell_type_colors', 'gene_embedding_path', 'hvg', 'log1p', 'neighbors', 'rank_genes_groups_cov', 'source_colors', 'split_1_colors', 'split_2_colors', 'split_3_colors', 'split_4_colors', 'split_5_colors', 'split_hardest_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    layers: 'counts'
    obsp: 'connectivities', 'd

In [3]:
adata.obs['condition'] = adata.obs['cond_harm'].copy().replace({'ctrl': 'control'})
adata.obs['condition'].value_counts()

/tmp/ipykernel_256685/1882561309.py:1: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  adata.obs['condition'] = adata.obs['cond_harm'].copy().replace({'ctrl': 'control'})


condition
control          11855
KLF1              1960
BAK1              1457
CEBPE             1233
CEBPE+RUNX1T1     1219
                 ...  
FOSB+CEBPB          71
CEBPB+CEBPA         64
CBL+UBASH3A         64
JUN+CEBPB           59
JUN+CEBPA           54
Name: count, Length: 235, dtype: int64

In [4]:
adata.obs['cell_type'] = adata.obs['cell_line'].copy().replace({
    'K562': 'k562'
})
adata.obs['cell_type'].value_counts()

/tmp/ipykernel_256685/2180255941.py:1: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  adata.obs['cell_type'] = adata.obs['cell_line'].copy().replace({


cell_type
k562    111122
Name: count, dtype: int64

In [5]:
split_cols = [x for x in adata.obs.columns if 'split' in x]
split_cols

['split_hardest',
 'split_1',
 'split_2',
 'split_3',
 'split_4',
 'split_5',
 'split_6']

In [ ]:
for split_col in split_cols:
    split = adata.obs[split_col].replace({
        'valid': 'val',
        'ood': 'test',
    }).copy()
    
    unique_train_perts = set()
    for perts in adata[split == 'train'].obs['condition'].unique():
        for pert in perts.split('+'):
            unique_train_perts.add(pert)
    
    perts_remove = []
    for perts in adata[split == 'val'].obs['condition'].unique():
        for pert in perts.split('+'):
            if pert not in unique_train_perts:
                perts_remove.append(perts)
    
    for perts in adata[split == 'test'].obs['condition'].unique():
        for pert in perts.split('+'):
            if pert not in unique_train_perts:
                perts_remove.append(perts)
    
    print(split_col, len(perts_remove))
    perts_remove_idx = adata[adata.obs.condition.isin(perts_remove)].obs_names
    split.loc[perts_remove_idx] = None
    
    n_train = adata[split == 'train'].obs.condition.nunique()
    n_val = adata[split == 'val'].obs.condition.nunique()
    n_test = adata[split == 'test'].obs.condition.nunique()
    print(f'{split_col}: {n_train} train, {n_val} val, {n_test} test')
        
    train_control_cells = adata[
        (adata.obs[split_col] == 'train') &
        (adata.obs.condition == 'control')
    ].obs.index.tolist()
    test_control_cells = random.sample(train_control_cells, 1500)
    split.loc[test_control_cells] = 'test'
    
    out_path = f'{data_path}/norman19_cpa_hvg_normalized_splits/{split_col}.csv'
    split.to_csv(out_path, header=False)
    print(f'Saved to {out_path}')
    
    

In [ ]:
adata.X = csr_matrix(adata.X)
adata.write_h5ad(f'{data_path}/norman19_cpa_hvg_normalized_curated.h5ad')